In [75]:
using ZipFile, Distributed, CSV, DataFrames, Random, StatsBase, Plots, Statistics, StableRNGs, MLJBase
using Flux
using Flux: logitbinarycrossentropy
using BSON: @save
using Distributions
using PyCall
PyCall.libpython

file = CSV.read("train.csv", DataFrame)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Float64
1,842,0,2.2,0,1,0,7,0.6
2,1021,1,0.5,1,0,1,53,0.7
3,563,1,0.5,1,2,1,41,0.9
4,615,1,2.5,0,0,0,10,0.8
5,1821,1,1.2,0,13,1,44,0.6
6,1859,0,0.5,1,3,0,22,0.7
7,1821,0,1.7,0,4,1,10,0.8
8,1954,0,0.5,1,0,0,24,0.8
9,1445,1,0.5,0,0,0,53,0.7


In [76]:
# Transfer DataFrame to matrix form
df = file |> Tables.matrix

# Transfer the dataset to 2-classifiers. df_0 represents the result is 0, df_1 represents the result is 1.
# Due to my datasset is binary problems, and each result is 50 percents of the whole dataset. So i didn't add any other pre-actions for dataset. 

mat_0 = df[df[:,21] .<= 1, :]
mat_1 = df[df[:,21] .>= 2, :]


1000×21 Matrix{Float64}:
 1021.0  1.0  0.5  1.0   0.0  1.0  53.0  …   3.0   7.0  1.0  1.0  0.0  2.0
  563.0  1.0  0.5  1.0   2.0  1.0  41.0      2.0   9.0  1.0  1.0  0.0  2.0
  615.0  1.0  2.5  0.0   0.0  0.0  10.0      8.0  11.0  1.0  0.0  0.0  2.0
 1821.0  0.0  1.7  0.0   4.0  1.0  10.0      8.0  18.0  1.0  0.0  1.0  3.0
  769.0  1.0  2.9  1.0   0.0  0.0   9.0      2.0   7.0  0.0  0.0  0.0  3.0
 1520.0  1.0  2.2  0.0   5.0  1.0  33.0  …   9.0  13.0  1.0  1.0  1.0  3.0
  803.0  1.0  2.1  0.0   7.0  0.0  17.0      1.0   4.0  1.0  0.0  1.0  2.0
  838.0  0.0  0.5  0.0   1.0  1.0  13.0      9.0  19.0  1.0  0.0  1.0  3.0
  595.0  0.0  0.9  1.0   7.0  1.0  23.0      2.0  18.0  1.0  1.0  0.0  3.0
  772.0  0.0  1.1  1.0  12.0  0.0  39.0     15.0   3.0  1.0  1.0  0.0  3.0
    ⋮                         ⋮          ⋱   ⋮                         ⋮
 1784.0  0.0  1.6  0.0   4.0  0.0  41.0      1.0  11.0  0.0  1.0  0.0  2.0
 1262.0  0.0  1.8  1.0  12.0  0.0  34.0      3.0   4.0  0.0  1.0  1.0  2.0
 1

In [77]:
# randomly select 70% data from data set as the train data

train_data = randsubseq(1:1000, 0.7)
train_df = vcat(mat_0[train_data, :], mat_1[train_data, :])

# select the rest as the test data 

test_data = [i for i in 1:1000 if isempty(searchsorted(train_data, i))]
test_df = vcat(mat_0[test_data, :], mat_1[test_data, :])

560×21 Matrix{Float64}:
 1954.0  0.0  0.5  1.0   0.0  0.0  24.0  …   3.0   5.0  1.0  1.0  1.0  0.0
 1815.0  0.0  2.8  0.0   2.0  0.0  33.0      0.0   2.0  1.0  0.0  0.0  1.0
  682.0  1.0  0.5  0.0   4.0  0.0  19.0      1.0  18.0  0.0  1.0  1.0  1.0
  851.0  0.0  0.5  0.0   3.0  0.0  21.0      7.0  10.0  1.0  0.0  1.0  0.0
 1253.0  1.0  0.5  1.0   5.0  1.0   5.0      0.0   4.0  1.0  1.0  0.0  1.0
 1656.0  0.0  1.0  0.0   5.0  1.0  34.0  …  12.0  14.0  1.0  1.0  0.0  1.0
  823.0  1.0  2.7  1.0  13.0  0.0  60.0     11.0  17.0  1.0  1.0  1.0  0.0
 1484.0  0.0  3.0  0.0   3.0  0.0  12.0      4.0  20.0  1.0  1.0  0.0  1.0
 1242.0  0.0  1.1  1.0   0.0  0.0  10.0      1.0   4.0  1.0  0.0  1.0  0.0
  819.0  0.0  0.6  1.0   8.0  1.0  42.0      4.0  13.0  1.0  0.0  1.0  1.0
    ⋮                         ⋮          ⋱   ⋮                         ⋮
 1522.0  0.0  2.2  0.0   4.0  0.0  21.0      9.0  16.0  0.0  0.0  1.0  3.0
  757.0  1.0  0.9  1.0   4.0  0.0  64.0      5.0  10.0  1.0  1.0  0.0  2.0
 19

In [78]:
# parameters of the training and testing data
para_train = train_df[:, 1:20]
para_test = test_df[:, 1:20]

# value is never used?
value_train = train_df[:, 21]
value_test = test_df[:, 21]
value_train_std = reshape(value_train, (1, :)) 
value_test_std = reshape(value_test, (1, :)) 


1×560 Matrix{Float64}:
 0.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  …  3.0  3.0  3.0  3.0  2.0  3.0  3.0

In [79]:
dt = StatsBase.fit(ZScoreTransform, para_train, dims=1)

# rotate the matrix for 90 degree
para_train_std = StatsBase.transform(dt, para_train)
para_test_std = StatsBase.transform(dt, para_test)

para_train_std = transpose(para_train_std) 
para_test_std = transpose(para_test_std)

20×560 transpose(::Matrix{Float64}) with eltype Float64:
  1.65044    1.33224    -1.26143    …   1.67563     1.55201   -1.65517
 -1.00104   -1.00104     0.998265       0.998265   -1.00104    0.998265
 -1.23535    1.55995    -1.23535        1.31688    -0.749214   0.587673
  0.97768   -1.02212    -1.02212        0.97768     0.97768    0.97768
 -1.00533   -0.544432   -0.0835371     -1.00533    -0.774879   0.14691
 -1.04076   -1.04076    -1.04076    …  -1.04076     0.960168   0.960168
 -0.441525   0.0513826  -0.715363       0.379988    0.215685   0.708593
  1.06474    0.366074    1.76341       -1.03127     0.715409   1.41408
  1.33651    0.544876   -0.529483       1.33651    -0.897027   0.79933
 -0.215415  -0.215415   -0.215415      -0.215415    1.52238    0.653485
 -1.66686    1.15856     0.161353   …  -1.16825    -1.16825    0.992358
 -0.293542  -0.0791955   0.586407       0.615739    0.509694  -0.358974
 -0.231907  -1.15837    -0.42829        1.65336     0.884005  -1.14451
 -1.31408   -

In [80]:
# import xcs package
a = pyimport("xcs")

PyObject <module 'xcs' from '/Users/yuchen/.julia/conda/3/lib/python3.9/site-packages/xcs/__init__.py'>

In [81]:
#set variables
PreClassifiedData = a.scenarios.PreClassifiedData
XCSAlgorithm = a.XCSAlgorithm

PyObject <class 'xcs.XCSAlgorithm'>

In [82]:
# we'll create the algorithm which will be used to manage the classifier set and learn the mapping defined by the problem we have selected
al = XCSAlgorithm()

PyObject <xcs.XCSAlgorithm object at 0x1ac9a8340>

In [83]:
al.exploration_probability = .1
al.discount_factor = 0
al.do_ga_subsumption = true
al.do_action_set_subsumption = true

true

In [84]:
#output the model using py style
py"""
def outputmodel(x):
    import logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info('Classifiers:\n\n%s\n', x)

"""

In [85]:
scenarioTrain = PreClassifiedData(para_train,value_train)
scenarioTest = PreClassifiedData(para_test,value_test)

PyObject <xcs.scenarios.PreClassifiedData object at 0x1ac9db8b0>

In [86]:
model_train = al.new_model(scenarioTrain)
model_test = al.new_model(scenarioTest)

PyObject <xcs.ClassifierSet object at 0x1aca802e0>

In [87]:
#run the model
# model_train = al.run(scenarioTrain)
# model_test = al.run(scenarioTest)
model_train.run(scenarioTrain, learn = true)
model_test.run(scenarioTest, learn = true)

PyObject <xcs.ClassifierSet object at 0x1ac9aa430>

In [88]:
#train model output
py"outputmodel"(model_train)

INFO:__main__:Classifiers:

###110#111##11101111 => 3.0
    Time Stamp: 83
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#0#1#01#111111##1011 => 3.0
    Time Stamp: 196
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1011#11#1#011##11##0 => 0.0
    Time Stamp: 202
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
111000##1#01#11111#1 => 3.0
    Time Stamp: 206
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
11100111110111111##1 => 0.0
    Time Stamp: 227
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#011#0111#111#10###0 => 0.0
    Time Stamp: 326
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0


In [89]:
#test model output
py"outputmodel"(model_test)

INFO:__main__:Classifiers:

#111011#1#0#1111#101 => 2.0
    Time Stamp: 128
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
##101##11111#1#01##0 => 1.0
    Time Stamp: 131
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1#100#1#111##11##101 => 0.0
    Time Stamp: 154
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10101#11#1111#101### => 2.0
    Time Stamp: 175
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
111##11##11##1#0##00 => 2.0
    Time Stamp: 234
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1#1#011###11#1#11#00 => 2.0
    Time Stamp: 239
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0